In [18]:
import gymnasium as gym
import torch
from gymnasium.wrappers import RescaleAction

In [19]:
env = gym.make("Pendulum-v1", render_mode="rgb_array")

In [20]:
env.reset()

(array([-0.6064908 ,  0.79509056, -0.33612338], dtype=float32), {})

In [23]:
env.unwrapped.state

array([ 2.22243582, -0.33612338])

In [27]:
min_action, max_action = -1, 1
env = RescaleAction(env, min_action, max_action)

In [28]:
env.unwrapped.state

array([ 2.22243582, -0.33612338])

In [29]:
class RewardWrapper(gym.Env):
    def __init__(self, reward_fn):
        super().__init__()
        self.reward_fn = reward_fn

    def reward(self, reward):
        state = torch.tensor(self.env.unwrapped.state, dtype=torch.float32)
        action = torch.tensor(self.env.unwrapped.last_action, dtype=torch.float32)
        with torch.no_grad():
            custom_reward = self.reward_fn(state, action).item()
        return custom_reward

In [30]:
wrapped_env = RewardWrapper(env, lambda x, y: 0)

In [31]:
wrapped_env.reset()

(array([-0.54628414, -0.83759993,  0.7744711 ], dtype=float32), {})

In [32]:
wrapped_env.step(1)

AttributeError: 'RescaleAction' object has no attribute 'state'